# 数据准备

In [1]:
from datasets import load_dataset

data = load_dataset("cardiffnlp/tweet_sentiment_multilingual/data/english")
data["val"] = data["validation"]
del data["validation"]

In [2]:
import pandas as pd
data["train"].to_pandas().label.value_counts()

label
0    613
1    613
2    613
Name: count, dtype: int64

In [3]:
# 字符数
max_char = data['train'].to_pandas()['text'].str.len().max()
print(f"The maximum number of characters is {max_char}")
# 词数
max_words = data['train'].to_pandas()['text'].str.split().str.len().max()
print(f"The maximum number of words is {max_words}")

The maximum number of characters is 185
The maximum number of words is 33


## llama tokenizer

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding
llama_path = "./Meta-Llama-3.1-8B"
llama_tokenizer = AutoTokenizer.from_pretrained(llama_path, add_prefix_space=True)
llama_tokenizer.pad_token_id = llama_tokenizer.eos_token_id
llama_tokenizer.pad_token = llama_tokenizer.eos_token
def preprocessing_function(examples):
    examples['label'] = [int(i) for i in examples['label']]
    return llama_tokenizer(examples['text'], truncation=True, padding='max_length', max_length=64)

tokenized_data = data.map(preprocessing_function, batched=True, remove_columns= ["text"])
tokenized_data.set_format("torch")
# 创建 DataCollatorWithPadding 实例
data_collator = DataCollatorWithPadding(tokenizer=llama_tokenizer)


In [5]:
tokenized_data["train"]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 1839
})

## llama model

In [6]:
from transformers import AutoModelForSequenceClassification
import torch

pretrain_model = AutoModelForSequenceClassification.from_pretrained(llama_path, 
                                                                 num_labels=3,
                                                                device_map="auto",
                                                                offload_folder="offload",
                                                                trust_remote_code=True)
pretrain_model.config.pad_token_id = llama_tokenizer.pad_token_id
# llama_model.config.use_cache = False
# llama_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ./Meta-Llama-3.1-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model_dtype = next(pretrain_model.parameters()).dtype
print(f"Model dtype: {model_dtype}")

Model dtype: torch.float32


## LoRA 配置

In [24]:
from peft import get_peft_model, LoraConfig, TaskType
lora_alpha = 8
lora_rank = 4


lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    r=lora_rank, 
    lora_alpha=lora_alpha, 
    lora_dropout=0.05, 
    bias="none",
    target_modules=[
       "q_proj",  "v_proj"
    ],
)

llama_model = get_peft_model(pretrain_model, lora_config)
llama_model.print_trainable_parameters()

trainable params: 6,959,104 || all params: 7,511,896,064 || trainable%: 0.0926


## 评估指标

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # eval_pred 是模型返回的预测值和实际值元组
    predictions = np.argmax(logits, axis=-1)
    
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')
    accuracy = accuracy_score(labels, predictions)
    
    # 返回包含所有指标的字典
    return {"precision": precision, "recall": recall, "f1-score": f1, "accuracy": accuracy}


In [26]:
from transformers import TrainingArguments, Trainer

lr = 1e-4
batch_size = 8
num_epochs = 5

training_args = TrainingArguments(
    output_dir=f"./results/lr={lr}_lora_alpha={lora_alpha}_lora_r={lora_rank}",
    learning_rate=lr,
    lr_scheduler_type= "constant",
    warmup_ratio= 0.1,
    max_grad_norm= 0.3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.001,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, 
    load_best_model_at_end=True,
    report_to="wandb",
    fp16=True,
    gradient_checkpointing=True,
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
trainer = Trainer(
    model=pretrain_model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['val'],
    tokenizer=llama_tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [23]:
# 开始训练

trainer.train()

/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
1,No log,0.699357,0.717789,0.706790,0.710190,0.706790
2,No log,0.833159,0.685659,0.688272,0.655947,0.688272
3,0.871600,0.751952,0.761451,0.762346,0.761073,0.762346
4,0.871600,1.516374,0.711256,0.709877,0.708075,0.709877
5,0.272300,2.267151,0.710639,0.706790,0.704869,0.706790


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autoca

TrainOutput(global_step=1150, training_loss=0.5121667812181556, metrics={'train_runtime': 631.7992, 'train_samples_per_second': 14.554, 'train_steps_per_second': 1.82, 'total_flos': 2.466870316498944e+16, 'train_loss': 0.5121667812181556, 'epoch': 5.0})

In [10]:
from transformers import TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType

batch_size = 8
num_epochs = 10
lr_pars = [1e-05]
lora_ranks = [4,8,16,32]
lora_alphas = [16,32,64]

for lr in lr_pars:
    for lora_rank in lora_ranks:
        for lora_alpha in lora_alphas:
            
            lora_config = LoraConfig(
                task_type=TaskType.SEQ_CLS, 
                r=lora_rank, 
                lora_alpha=lora_alpha, 
                lora_dropout=0.05, 
                bias="none",
                target_modules=[
                "q_proj",  "v_proj"
                ],
            )

            llama_model = get_peft_model(pretrain_model, lora_config)
            llama_model.print_trainable_parameters()

            training_args = TrainingArguments(
                output_dir=f"./results/lr={lr}_lora_alpha={lora_alpha}_lora_r={lora_rank}",
                learning_rate=lr,
                lr_scheduler_type= "constant",
                warmup_ratio= 0.1,
                max_grad_norm= 0.3,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                num_train_epochs=num_epochs,
                weight_decay=0.001,
                eval_strategy="epoch",
                save_strategy="epoch",
                metric_for_best_model="eval_f1-score",  # 使用准确率选择最佳模型
                greater_is_better=True,       # 指标越大越好
                save_total_limit=3, 
                load_best_model_at_end=True,
                report_to="wandb",
                fp16=True,
                gradient_checkpointing=True,
            )

            trainer = Trainer(
                model=llama_model,
                args=training_args,
                train_dataset=tokenized_data['train'],
                eval_dataset=tokenized_data['val'],
                tokenizer=llama_tokenizer,
                compute_metrics=compute_metrics,
                data_collator=data_collator
            )

            # 开始训练
            print(f"------lr={lr}_lora_rank={lora_rank}_lora_alpha={lora_alpha}")
            trainer.train()     
            trainer.save_model(f"./results/lr={lr}_lora_alpha={lora_alpha}_lora_r={lora_rank}/best_model")
            



/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


trainable params: 1,716,224 || all params: 7,506,653,184 || trainable%: 0.0229


------lr=0.0002_lora_rank=4_lora_alpha=16


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zhenyiq (zhenyiq-federal-student-aid). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


...

/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


------lr=1e-05_lora_rank=32_lora_alpha=64


/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
1,No log,1.170840,0.441768,0.441358,0.435740,0.441358
2,No log,0.994812,0.515999,0.524691,0.513625,0.524691
3,1.266100,0.905297,0.589179,0.589506,0.588464,0.589506
4,1.266100,0.899075,0.615679,0.595679,0.600722,0.595679
5,0.680800,0.982275,0.589820,0.583333,0.583537,0.583333


/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/vipuser/anaconda3/envs/ec/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should 